### Merge Parking Ticket Data

Function to merge Los Angeles Parking Ticket data, without duplicates.

[Parking Tickets API](https://dev.socrata.com/foundry/data.lacity.org/4f5p-udkv)

[Paging through Data](https://dev.socrata.com/docs/paging)

In [15]:
import requests
import json
from config import get_api_token
import pandas as pd

In [16]:
# token, adjust tickets limits

app_token = get_api_token()
print(app_token)
base_url = 'https://data.lacity.org/resource/4f5p-udkv.json'

# get headers
headers = {
    'Accept': 'application/json',  
    'X-App-Token': app_token  
}
    
# Define the schema with default values
schema = {
    'agency': None,
    'agency_desc': None,
    'body_style': None,
    'body_style_desc': None,
    'color': None,
    'color_desc': None,
    'fine_amount': None,
    'issue_date': None,
    'issue_time': None,
    'loc_lat': None,
    'loc_long': None,
    'location': None,
    'make': None,
    'marked_time': None,
    'meter_id': None,
    'plate_expiry_date': None,
    'rp_state_plate': None,
    'ticket_number': None,
    'violation_code': None,
    'violation_description': None
}

# schema for data
def apply_schema(data, schema):
    """Apply the schema to individual data entries."""
    return {key: data.get(key, schema[key]) for key in schema}

# function to get initial data
def fetch_pages(base_url, limit, pages, schema, headers):
    all_data = []
    order = "issue_date ASC, issue_time ASC, ticket_number ASC"  # Ordering by date, time, then ticket number
    for page in range(pages):
        offset = limit * page
        url = f"{base_url}?$limit={limit}&$offset={offset}&$order={order}"
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            raw_data = response.json()
            # Apply the schema to each item in the data
            normalized_data = [apply_schema(item, schema) for item in raw_data]
            all_data.extend(normalized_data)
            print(f"Fetched page {page + 1} with {len(normalized_data)} records.")
        else:
            print(f"Failed to fetch data on page {page + 1}: {response.status_code}")
            break
    
    return all_data

def fetch_and_count_records(base_url, limit, page_count):
    all_data = []
    for page in range(page_count):
        offset = limit * page
        url = f"{base_url}?$limit={limit}&$offset={offset}"
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            all_data.extend(data) 
            print(f"Page {page + 1}: Fetched {len(data)} records.")
        else:
            print(f"Error fetching page {page + 1}: HTTP {response.status_code}")
            break  # exit the loop on error
    return all_data



ZzVI2vN6lETzguOojTmivo03L


In [17]:
# adding ticket data to existing data

def fetch_additional_records(base_url, limit, existing_data, schema, headers):
    if not existing_data:
        print("No existing data to determine the latest record.")
        return

    # Sort existing data by issue_date and ticket_number to find the latest one
    existing_data_sorted = sorted(existing_data, key=lambda x: (x['issue_date'], int(x['ticket_number'])))
    latest_record = existing_data_sorted[-1]
    
    # Build a condition to fetch records after the latest one
    where_condition = f"issue_date>'{latest_record['issue_date']}' OR (issue_date='{latest_record['issue_date']}' AND ticket_number>'{latest_record['ticket_number']}')"

    # Prepare the URL with the where condition
    url = f"{base_url}?$limit={limit}&$where={where_condition}&$order=issue_date ASC, ticket_number ASC"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        new_data = response.json()
        if not new_data:
            print("No new data found.")
            return

        # Apply the schema to each incoming record
        normalized_new_data = [apply_schema(record, schema) for record in new_data]
        
        # Merge new records
        existing_data.extend(normalized_new_data)
        print(f"Added {len(normalized_new_data)} new records to the existing data.")
    else:
        print(f"Failed to fetch data: {response.status_code}")



In [18]:
import pprint
limit = 1050  # records per page
pages = 1  # pages to fetch
order =  "ticket_number"
data = fetch_pages(base_url, limit, pages, schema, headers)

print(f"Total records fetched: {len(data)}")


existing = (data[:100])  # test set of records

Fetched page 1 with 1050 records.
Total records fetched: 1050


In [19]:
existing

[{'agency': '2',
  'agency_desc': 'LAX CURRENT',
  'body_style': 'PA',
  'body_style_desc': 'PASSENGER CAR',
  'color': 'WH',
  'color_desc': None,
  'fine_amount': None,
  'issue_date': '1950-07-15T00:00:00.000',
  'issue_time': '1550',
  'loc_lat': '33.9430863',
  'loc_long': '-118.3995274',
  'location': '701 WORLD WAY',
  'make': 'LAND',
  'marked_time': None,
  'meter_id': None,
  'plate_expiry_date': '201910',
  'rp_state_plate': 'CA',
  'ticket_number': '1127488375',
  'violation_code': None,
  'violation_description': None},
 {'agency': '1',
  'agency_desc': 'WESTERN',
  'body_style': 'SU',
  'body_style_desc': None,
  'color': 'BK',
  'color_desc': 'BLACK',
  'fine_amount': None,
  'issue_date': '1962-01-30T00:00:00.000',
  'issue_time': '1650',
  'loc_lat': '34.0337275',
  'loc_long': '-118.2653638',
  'location': '8TH ST W/O HILL ST',
  'make': 'JEEP',
  'marked_time': None,
  'meter_id': 'CB3235',
  'plate_expiry_date': '202005',
  'rp_state_plate': 'CA',
  'ticket_number':

In [20]:
# merging ticket data 
limit = 100

# Add more rows, apply merge function
fetch_additional_records(base_url, limit, existing, schema, headers)

Added 100 new records to the existing data.


In [21]:
print(existing[0])

{'agency': '2', 'agency_desc': 'LAX CURRENT', 'body_style': 'PA', 'body_style_desc': 'PASSENGER CAR', 'color': 'WH', 'color_desc': None, 'fine_amount': None, 'issue_date': '1950-07-15T00:00:00.000', 'issue_time': '1550', 'loc_lat': '33.9430863', 'loc_long': '-118.3995274', 'location': '701 WORLD WAY', 'make': 'LAND', 'marked_time': None, 'meter_id': None, 'plate_expiry_date': '201910', 'rp_state_plate': 'CA', 'ticket_number': '1127488375', 'violation_code': None, 'violation_description': None}


In [22]:
df = pd.DataFrame(existing)
display(df)

,agency,agency_desc,body_style,body_style_desc,color,color_desc,fine_amount,issue_date,issue_time,loc_lat,loc_long,location,make,marked_time,meter_id,plate_expiry_date,rp_state_plate,ticket_number,violation_code,violation_description
0,2,LAX CURRENT,PA,PASSENGER CAR,WH,None,None,1950-07-15T00:00:00.000,1550,33.9430863,-118.3995274,701 WORLD WAY,LAND,None,None,201910,CA,1127488375,None,None
1,1,WESTERN,SU,None,BK,BLACK,None,1962-01-30T00:00:00.000,1650,34.0337275,-118.2653638,8TH ST W/O HILL ST,JEEP,None,CB3235,202005,CA,1121858091,None,None
2,1,WESTERN,PA,PASSENGER CAR,BK,BLACK,None,1970-01-31T00:00:00.000,1840,34.0367254,-118.2682253,VENICE-PEMBROKE,HOND,None,None,202003,CA,1121875510,None,None
3,1,WESTERN,PA,PASSENGER CAR,GY,GREY,None,1972-01-22T00:00:00.000,1555,34.1564506,-118.4391616,14000 RIVERSIDE DR,TOYO,None,None,202009,CA,1121270581,None,None
4,1,WESTERN,PA,PASSENGER CAR,TA,None,None,1972-01-30T00:00:00.000,1805,34.0451361,-118.256675,8TH ST W/O OLIVE,TOYO,None,None,202004,CA,1121858124,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1,WESTERN,PA,PASSENGER CAR,YE,YELLOW,73,2012-07-18T00:00:00.000,2330,34.0384125,-118.230275,6TH ST BRIDGE,NISS,0000,None,0,CA,1120692985,8069B,None
196,4,HOLLYWOOD,PA,PASSENGER CAR,GY,GREY,63,2012-07-20T00:00:00.000,2308,34.123247,-118.302221,2800 W OBSERVATORY,TOYO,None,None,201712,CA,1117027354,8603,PK IN PROH AREA
197,2,LAX CURRENT,TR,TRAILER,WH,None,53,2012-07-29T00:00:00.000,2250,33.9442025,-118.4078454,300 WORLD WAY L/L,None,None,None,201710,CA,1112469864,8939,WHITE CURB
198,53,53 - DOT - VALLEY,PA,PASSENGER CAR,MA,None,68,2012-08-04T00:00:00.000,1212,34.1631709,-118.6044432,5141 DON PIO DR,LEXU,None,None,201510,CA,1111255456,8069BS,NO PARK/STREET CLEAN


In [23]:
# Check for duplicate rows
duplicate_rows = df.duplicated()

print(df[duplicate_rows])

Empty DataFrame
Columns: [agency, agency_desc, body_style, body_style_desc, color, color_desc, fine_amount, issue_date, issue_time, loc_lat, loc_long, location, make, marked_time, meter_id, plate_expiry_date, rp_state_plate, ticket_number, violation_code, violation_description]
Index: []
